In [1]:
import pandas as pd
import numpy as np
import time, os
os.chdir('..')
import functions.myfunctions as mf
# import functions.IPAfunctions as ipa
from sklearn.metrics import mutual_info_score as MI
import tidytcells as tt

MI between cdr3s

In [6]:
def cdr3_mi(a, b):
    mi = np.zeros(shape=(a.shape[1],b.shape[1]))                
    for i in range(a.shape[1]):
        for j in range(b.shape[1]):
            mi[i,j] = MI(a[:,i], b[:,j])
    
    return(mi)

In [ ]:
def save_cdr3_MI(mydict):
    mutualInfo = {}
    # print(mydict)
    # I want to save both the matrix and the sum
    for key in mydict.keys(): # either ep or ep + '_shuffle'
        mutualInfo[key] = {}
        for ss in mydict[key].keys():
            # and calculate the sum of each
            sumlist = [x.sum() for x in mydict[key][ss]]
            mutualInfo[key][ss] = sumlist
    r = pd.DataFrame(mutualInfo).T
    r['type'] = 'cdr3a-cdr3b'
    
    ep = set([k.split('_')[0] for k in mydict])
    assert len(ep) == 1
    ep = list(ep)[0]

    midf = pd.DataFrame()

    # save all arrays
    for i in mydict[ep].keys():
        assert len(mydict[ep][i]) == 10
        mean_array = np.stack([np.array(x).ravel() for x in mydict[ep][i]], axis=0)
        mean_shuffle = np.stack([np.array(x).ravel() for x in mydict[ep + '_shuffle'][i]], axis=0)
        assert mean_array.shape == (len(mydict[ep][i]), mydict[ep][i][0].shape[0]*mydict[ep][i][0].shape[1])
        assert mean_shuffle.shape == (len(mydict[ep][i]), mydict[ep + '_shuffle'][i][0].shape[0]*mydict[ep + '_shuffle'][i][0].shape[1])

        meandf = pd.DataFrame(mean_array)
        meandf['shuffle'] = 'real'
        meandf['subsample'] = i
        meandf['epitope'] = ep
        
        shuffledf = pd.DataFrame(mean_shuffle)
        shuffledf['shuffle'] = 'shuffle'
        shuffledf['subsample'] = i
        shuffledf['epitope'] = ep
        
        midf = pd.concat([midf, meandf, shuffledf])

    midf.to_csv('data/output/mutual_info/mutual_info_cdr3ab_array_' + ep + '_subsamples_endpadding.csv')

    return(r)

MI with V, J etc

In [10]:
def cdr3_with_cat_MI(cat_series, s):
    mi = []
    for i in range(s.shape[1]):
        mi.append(MI(cat_series, s[:,i]))
    return(np.array(mi))

In [11]:
def save_gene_MI(mydict, R, typestring):

    r = pd.DataFrame(mydict).T
    r['type'] = typestring
    R = pd.concat([R, r])

    return(R)

In [12]:
def save_gene_cdr3_MI(mydict, R, MIDF, typestring, epitope):
    mutualInfo = {}
    # I want to save both the matrix and the sum
    for key in mydict.keys(): # either ep or ep + '_shuffle'
        mutualInfo[key] = {}
        for ss in mydict[key].keys():
            # and calculate the sum of each
            sumlist = [x.sum() for x in mydict[key][ss]]
            mutualInfo[key][ss] = sumlist
    r = pd.DataFrame(mutualInfo).T
    r['type'] = typestring
    R = pd.concat([R, r])

    ep = set([k.split('_')[0] for k in mydict])
    assert len(ep) == 1
    ep = list(ep)[0]
    # save all array
    for ss in mydict[ep].keys():
        assert len(mydict[ep][ss]) == 10
        mean_array = np.stack([np.array(x).ravel() for x in mydict[ep][ss]], axis=0)
        mean_shuffle = np.stack([np.array(x).ravel() for x in mydict[ep + '_shuffle'][ss]], axis=0)
        assert mean_array.shape == (len(mydict[ep][ss]), mydict[ep][ss][0].shape[0])
        assert mean_shuffle.shape == (len(mydict[ep + '_shuffle'][ss]), mydict[ep + '_shuffle'][ss][0].shape[0])

        meandf = pd.DataFrame(mean_array)
        meandf['epitope'] = epitope
        meandf['subsample'] = ss
        meandf_shuffle = pd.DataFrame(mean_shuffle)
        meandf_shuffle['epitope'] = epitope
        meandf_shuffle['shuffle'] = 'shuffle'
        meandf_shuffle['subsample'] = ss
        MIDF = pd.concat([MIDF, meandf, meandf_shuffle])

    return(R, MIDF)

In [17]:
tanno = pd.read_csv('data/tanno_A1naive_subset_for_MI.csv', index_col=0)
print(tanno.head())
# get only cdr3s of specific length
tanno['cdr3a'] = tanno['cdr3a_IMGTgaps'].str.replace('-','')
tanno['len_cdr3a_nogaps'] = [len(x) for x in tanno['cdr3a']]
tanno['cdr3b'] = tanno['cdr3b_IMGTgaps'].str.replace('-','')
tanno['len_cdr3b_nogaps'] = [len(x) for x in tanno['cdr3b']]


tanno['cdr3a_endpadding'] = [x + ''.join(['-']*(19-len(x))) for x in tanno['cdr3a']]
tanno['cdr3b_endpadding'] = [x + ''.join(['-']*(19-len(x))) for x in tanno['cdr3b']]

print(tanno.shape)
tanno = tanno.loc[(tanno['len_cdr3a_nogaps'] == 13) & (tanno['len_cdr3b_nogaps'] == 15)]
print(tanno.shape)
epset = mf.prepare_data(tanno, col1 = 'cdr3a_endpadding', col2='cdr3b_endpadding', type = 'cdr3')
print(tanno.shape)
ind = 'Tanno::A1::naive'

print(tanno['cdr3a_endpadding'])
print(tanno['cdr3b_endpadding'])

assert all([len(x) == 19 for x in tanno['cdr3a_endpadding']])
assert all([len(x) == 19 for x in tanno['cdr3b_endpadding']])

   Clustered  Collapsed  ClusterID           CDRH3_AA        CDRL3_AA  \
0         60         11         97  CASSQDLSRLASYEQYF   CAVSDLGFQKLVF   
2         58          3       7026     CSVEDHGSYNEQFF  CIVRVFTNAGKSTF   
1         58          4       2250      CASSEIGIQPQHF    CLASGSWGKFQF   
3         56         26          1     CATSSLAGVETQYF  CIVRAVQGAQKLVF   
4         56         13         44   CASSQGQGARENIQYF   CAALSGGSYIPTF   

                                            CDRH3_NT  \
0  TGCGCCAGCAGCCAAGACCTGAGCCGGCTAGCCTCCTACGAGCAGT...   
2         TGCAGCGTTGAAGATCACGGCTCCTACAATGAGCAGTTCTTC   
1            TGTGCCAGCAGTGAAATTGGAATTCAGCCCCAGCATTTT   
3         TGTGCCACCAGCAGTCTAGCGGGAGTCGAGACCCAGTACTTC   
4   TGCGCCAGCAGCCAAGGACAGGGAGCGAGAGAAAACATTCAGTACTTC   

                                     CDRL3_NT        VH     DH       JH  \
0     TGTGCTGTGAGTGATTTAGGCTTTCAGAAACTTGTATTT   TRBV4-1  TRBD2  TRBJ2-7   
2  TGCATCGTCAGAGTGTTTACCAATGCAGGCAAATCAACCTTT  TRBV29-1    NaN  TRBJ2-1   

In [18]:
for c in ['VL', 'VH', 'JL', 'JH']:
    print(c)
    epset[c] = epset.apply(
        lambda row: pd.NA if type(row[c]) != str else tt.tcr.standardise(
            gene_name=row[c],
            species='HomoSapiens',
            precision='gene'
        ),
        axis=1
    )
epset00 = epset.dropna(subset=['VL', 'VH', 'JL', 'JH'], how='any')
print('shape before tidytcells: ', epset.shape, '; shape after tidytcells: ', epset00.shape)
# tanno = tanno00.copy()

VL
VH
JL
JH
shape before tidytcells:  (835, 30) ; shape after tidytcells:  (835, 30)


In [19]:
epset.columns

Index(['Clustered', 'Collapsed', 'ClusterID', 'CDRH3_AA', 'CDRL3_AA',
       'CDRH3_NT', 'CDRL3_NT', 'VH', 'DH', 'JH', 'VL', 'JL', 'subject',
       'sample', 'TCRa', 'TCRb', 'cdr3a_IMGTgaps', 'cdr3b_IMGTgaps',
       'len_cdr3a', 'len_cdr3b', 'cdr3a', 'len_cdr3a_nogaps', 'cdr3b',
       'len_cdr3b_nogaps', 'cdr3a_endpadding', 'cdr3b_endpadding',
       'len_cdr3a_endpadding', 'cdr3a_endpadding_padded',
       'len_cdr3b_endpadding', 'cdr3b_endpadding_padded'],
      dtype='object')

In [20]:
subsamples = [2, 5, 10, 15, 20, 25, 35, 50, 80, 100, 150, 200, 300, 500, 1000, 1500, 2000, 2500, 3000, 5000, 10000, 15000]
mutualInfo = {}

In [22]:
all_MI = {}

In [23]:
mutualInfo = {}
mutualInfo[ind] = {}
mutualInfo[ind + '_shuffle'] = {}
print(ind, epset.shape[0])
i = 0
s_size = subsamples[0]
while s_size < epset.shape[0]:
    print(s_size)
    mutualInfo[ind][s_size] = []
    mutualInfo[ind + '_shuffle'][s_size] = []
    # print(s_size)
    for j in range(10):
        # print(j)
        epset_sample = epset.sample(s_size)
        epset_sample1 = epset_sample.sample(s_size)
        a = np.array(epset_sample['cdr3a_endpadding_padded'].str.split(', ', expand=True))
        b = np.array(epset_sample['cdr3b_endpadding_padded'].str.split(', ', expand=True))
        b_s = np.array(epset_sample1['cdr3b_endpadding_padded'].str.split(', ', expand=True))

        mi = cdr3_mi(a,b)
        shuffle_mi = cdr3_mi(a, b_s)

        mutualInfo[ind][s_size].append(mi)
        mutualInfo[ind + '_shuffle'][s_size].append(shuffle_mi)
    i += 1
    s_size = subsamples[i]

# finally calculate on entire set

s_size = epset.shape[0]
mutualInfo[ind][s_size] = []
mutualInfo[ind + '_shuffle'][s_size] = []
for j in range(10):
    # putting also real calcs in here because I want 10 points for the fitting
    a = np.array(epset['cdr3a_endpadding_padded'].str.split(', ', expand=True))
    b = np.array(epset['cdr3b_endpadding_padded'].str.split(', ', expand=True))    
    mi = cdr3_mi(a,b)
    mutualInfo[ind][s_size].append(mi)
    epset_sample1 = epset.sample(epset.shape[0])
    b_s = np.array(epset_sample1['cdr3b_endpadding_padded'].str.split(', ', expand=True))
    mutualInfo[ind + '_shuffle'][s_size].append(cdr3_mi(a, b_s))

print(mutualInfo[ind].keys())
r = save_cdr3_MI(mutualInfo)
all_MI[ind] = r

Tanno::A1::naive 835
2


5
10
15
20
25
35
50
80
100
150
200
300
500
dict_keys([2, 5, 10, 15, 20, 25, 35, 50, 80, 100, 150, 200, 300, 500, 835])


Repeat with V, J etc

In [24]:
gene_cdr3_results = {'Va-CDR3a':pd.DataFrame(),
                     'Vb-CDR3b':pd.DataFrame(),
                     'Ja-CDR3a':pd.DataFrame(),
                     'Jb-CDR3b':pd.DataFrame(),
                     'Vb-CDR3a':pd.DataFrame(),
                     'Va-CDR3b':pd.DataFrame(),
                     'Jb-CDR3a':pd.DataFrame(),
                     'Ja-CDR3b':pd.DataFrame(),}

In [25]:
# for background

ep = ind

mutualInfo_VaVb = {ep:{}, ep+'_shuffle':{}}
mutualInfo_JaJb = {ep:{}, ep+'_shuffle':{}}
mutualInfo_VaJa = {ep:{}, ep+'_shuffle':{}}
mutualInfo_VbJb = {ep:{}, ep+'_shuffle':{}}
mutualInfo_VaJb = {ep:{}, ep+'_shuffle':{}}
mutualInfo_VbJa = {ep:{}, ep+'_shuffle':{}}
mutualInfo_VaCDR3a = {ep:{}, ep+'_shuffle':{}}
mutualInfo_JaCDR3a = {ep:{}, ep+'_shuffle':{}}
mutualInfo_VbCDR3a = {ep:{}, ep+'_shuffle':{}}
mutualInfo_JbCDR3a = {ep:{}, ep+'_shuffle':{}}
mutualInfo_VaCDR3b = {ep:{}, ep+'_shuffle':{}}
mutualInfo_JaCDR3b = {ep:{}, ep+'_shuffle':{}}
mutualInfo_VbCDR3b = {ep:{}, ep+'_shuffle':{}}
mutualInfo_JbCDR3b = {ep:{}, ep+'_shuffle':{}}

print(ep, epset.shape[0])
i = 0
s_size = subsamples[0]
while s_size < epset.shape[0]:
    print(s_size)
    mutualInfo_VaVb[ep][s_size] = []
    mutualInfo_JaJb[ep][s_size] = []
    mutualInfo_VaJa[ep][s_size] = []
    mutualInfo_VbJb[ep][s_size] = []
    mutualInfo_VaJb[ep][s_size] = []
    mutualInfo_VbJa[ep][s_size] = []
    mutualInfo_VaCDR3a[ep][s_size] = []
    mutualInfo_JaCDR3a[ep][s_size] = []
    mutualInfo_VbCDR3a[ep][s_size] = []
    mutualInfo_JbCDR3a[ep][s_size] = []
    mutualInfo_VaCDR3b[ep][s_size] = []
    mutualInfo_JaCDR3b[ep][s_size] = []
    mutualInfo_VbCDR3b[ep][s_size] = []
    mutualInfo_JbCDR3b[ep][s_size] = []
    
    mutualInfo_VaVb[ep+'_shuffle'][s_size] = []
    mutualInfo_JaJb[ep+'_shuffle'][s_size] = []
    mutualInfo_VaJa[ep+'_shuffle'][s_size] = []
    mutualInfo_VbJb[ep+'_shuffle'][s_size] = []
    mutualInfo_VaJb[ep+'_shuffle'][s_size] = []
    mutualInfo_VbJa[ep+'_shuffle'][s_size] = []
    mutualInfo_VaCDR3a[ep+'_shuffle'][s_size] = []
    mutualInfo_JaCDR3a[ep+'_shuffle'][s_size] = []
    mutualInfo_VbCDR3a[ep+'_shuffle'][s_size] = []
    mutualInfo_JbCDR3a[ep+'_shuffle'][s_size] = []
    mutualInfo_VaCDR3b[ep+'_shuffle'][s_size] = []
    mutualInfo_JaCDR3b[ep+'_shuffle'][s_size] = []
    mutualInfo_VbCDR3b[ep+'_shuffle'][s_size] = []
    mutualInfo_JbCDR3b[ep+'_shuffle'][s_size] = []

    # print(s_size)
    for j in range(10):
        # print(j)
        epset_sample = epset.sample(s_size)
        epset_sample1 = epset_sample.sample(s_size)
        a = np.array(epset_sample['cdr3a_endpadding_padded'].str.split(', ', expand=True))
        b = np.array(epset_sample['cdr3b_endpadding_padded'].str.split(', ', expand=True))
        a_s = np.array(epset_sample1['cdr3a_endpadding_padded'].str.split(', ', expand=True))
        b_s = np.array(epset_sample1['cdr3b_endpadding_padded'].str.split(', ', expand=True))

        mutualInfo_VaVb[ep][s_size].append(MI(epset_sample['VL'], epset_sample['VH']))
        mutualInfo_JaJb[ep][s_size].append(MI(epset_sample['JL'], epset_sample['JH']))
        mutualInfo_VaJa[ep][s_size].append(MI(epset_sample['VL'], epset_sample['JL']))
        mutualInfo_VbJb[ep][s_size].append(MI(epset_sample['VH'], epset_sample['JH']))
        mutualInfo_VaJb[ep][s_size].append(MI(epset_sample['VL'], epset_sample['JH']))
        mutualInfo_VbJa[ep][s_size].append(MI(epset_sample['VH'], epset_sample['JL']))
        mutualInfo_VaCDR3a[ep][s_size].append(cdr3_with_cat_MI(epset_sample['VL'], a))
        mutualInfo_JaCDR3a[ep][s_size].append(cdr3_with_cat_MI(epset_sample['JL'], a))
        mutualInfo_VbCDR3a[ep][s_size].append(cdr3_with_cat_MI(epset_sample['VH'], a))
        mutualInfo_JbCDR3a[ep][s_size].append(cdr3_with_cat_MI(epset_sample['JH'], a))
        mutualInfo_VaCDR3b[ep][s_size].append(cdr3_with_cat_MI(epset_sample['VL'], b))
        mutualInfo_JaCDR3b[ep][s_size].append(cdr3_with_cat_MI(epset_sample['JL'], b))
        mutualInfo_VbCDR3b[ep][s_size].append(cdr3_with_cat_MI(epset_sample['VH'], b))
        mutualInfo_JbCDR3b[ep][s_size].append(cdr3_with_cat_MI(epset_sample['JH'], b))

        mutualInfo_VaVb[ep+'_shuffle'][s_size].append(MI(epset_sample['VL'], epset_sample1['VH']))
        mutualInfo_JaJb[ep+'_shuffle'][s_size].append(MI(epset_sample['JL'], epset_sample1['JH']))
        mutualInfo_VaJa[ep+'_shuffle'][s_size].append(MI(epset_sample['VL'], epset_sample1['JL']))
        mutualInfo_VbJb[ep+'_shuffle'][s_size].append(MI(epset_sample['VH'], epset_sample1['JH']))
        mutualInfo_VaJb[ep+'_shuffle'][s_size].append(MI(epset_sample['VL'], epset_sample1['JH']))
        mutualInfo_VbJa[ep+'_shuffle'][s_size].append(MI(epset_sample['VH'], epset_sample1['JL']))
        mutualInfo_VaCDR3a[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['VL'], a_s))
        mutualInfo_JaCDR3a[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['JL'], a_s))
        mutualInfo_VbCDR3a[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['VH'], a_s))
        mutualInfo_JbCDR3a[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['JH'], a_s))
        mutualInfo_VaCDR3b[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['VL'], b_s))
        mutualInfo_JaCDR3b[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['JL'], b_s))
        mutualInfo_VbCDR3b[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['VH'], b_s))
        mutualInfo_JbCDR3b[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['JH'], b_s))

    i += 1
    s_size = subsamples[i]

# finally calculate on entire set
a = np.array(epset['cdr3a_endpadding_padded'].str.split(', ', expand=True))
b = np.array(epset['cdr3b_endpadding_padded'].str.split(', ', expand=True))    

epset_sample = epset.copy()
s_size = epset.shape[0]
mutualInfo_VaVb[ep][s_size] = []
mutualInfo_JaJb[ep][s_size] = []
mutualInfo_VaJa[ep][s_size] = []
mutualInfo_VbJb[ep][s_size] = []
mutualInfo_VaJb[ep][s_size] = []
mutualInfo_VbJa[ep][s_size] = []
mutualInfo_VaCDR3a[ep][s_size] = []
mutualInfo_JaCDR3a[ep][s_size] = []
mutualInfo_VbCDR3a[ep][s_size] = []
mutualInfo_JbCDR3a[ep][s_size] = []
mutualInfo_VaCDR3b[ep][s_size] = []
mutualInfo_JaCDR3b[ep][s_size] = []
mutualInfo_VbCDR3b[ep][s_size] = []
mutualInfo_JbCDR3b[ep][s_size] = []

mutualInfo_VaVb[ep+'_shuffle'][s_size] = []
mutualInfo_JaJb[ep+'_shuffle'][s_size] = []
mutualInfo_VaJa[ep+'_shuffle'][s_size] = []
mutualInfo_VbJb[ep+'_shuffle'][s_size] = []
mutualInfo_VaJb[ep+'_shuffle'][s_size] = []
mutualInfo_VbJa[ep+'_shuffle'][s_size] = []
mutualInfo_VaCDR3a[ep+'_shuffle'][s_size] = []
mutualInfo_JaCDR3a[ep+'_shuffle'][s_size] = []
mutualInfo_VbCDR3a[ep+'_shuffle'][s_size] = []
mutualInfo_JbCDR3a[ep+'_shuffle'][s_size] = []
mutualInfo_VaCDR3b[ep+'_shuffle'][s_size] = []
mutualInfo_JaCDR3b[ep+'_shuffle'][s_size] = []
mutualInfo_VbCDR3b[ep+'_shuffle'][s_size] = []
mutualInfo_JbCDR3b[ep+'_shuffle'][s_size] = []

for j in range(10):
    # putting also real calcs in here because I want 10 points for the fitting
    mutualInfo_VaVb[ep][s_size].append(MI(epset_sample['VL'], epset_sample['VH']))
    mutualInfo_JaJb[ep][s_size].append(MI(epset_sample['JL'], epset_sample['JH']))
    mutualInfo_VaJa[ep][s_size].append(MI(epset_sample['VL'], epset_sample['JL']))
    mutualInfo_VbJb[ep][s_size].append(MI(epset_sample['VH'], epset_sample['JH']))
    mutualInfo_VaJb[ep][s_size].append(MI(epset_sample['VL'], epset_sample['JH']))
    mutualInfo_VbJa[ep][s_size].append(MI(epset_sample['VH'], epset_sample['JL']))
    mutualInfo_VaCDR3a[ep][s_size].append(cdr3_with_cat_MI(epset_sample['VL'], a))
    mutualInfo_JaCDR3a[ep][s_size].append(cdr3_with_cat_MI(epset_sample['JL'], a))
    mutualInfo_VbCDR3a[ep][s_size].append(cdr3_with_cat_MI(epset_sample['VH'], a))
    mutualInfo_JbCDR3a[ep][s_size].append(cdr3_with_cat_MI(epset_sample['JH'], a))
    mutualInfo_VaCDR3b[ep][s_size].append(cdr3_with_cat_MI(epset_sample['VL'], b))
    mutualInfo_JaCDR3b[ep][s_size].append(cdr3_with_cat_MI(epset_sample['JL'], b))
    mutualInfo_VbCDR3b[ep][s_size].append(cdr3_with_cat_MI(epset_sample['VH'], b))
    mutualInfo_JbCDR3b[ep][s_size].append(cdr3_with_cat_MI(epset_sample['JH'], b))
    epset_sample1 = epset.sample(epset.shape[0])
    a_s = np.array(epset_sample1['cdr3a_endpadding_padded'].str.split(', ', expand=True))
    b_s = np.array(epset_sample1['cdr3b_endpadding_padded'].str.split(', ', expand=True))

    mutualInfo_VaVb[ep+'_shuffle'][s_size].append(MI(epset_sample['VL'], epset_sample1['VH']))
    mutualInfo_JaJb[ep+'_shuffle'][s_size].append(MI(epset_sample['JL'], epset_sample1['JH']))
    mutualInfo_VaJa[ep+'_shuffle'][s_size].append(MI(epset_sample['VL'], epset_sample1['JL']))
    mutualInfo_VbJb[ep+'_shuffle'][s_size].append(MI(epset_sample['VH'], epset_sample1['JH']))
    mutualInfo_VaJb[ep+'_shuffle'][s_size].append(MI(epset_sample['VL'], epset_sample1['JH']))
    mutualInfo_VbJa[ep+'_shuffle'][s_size].append(MI(epset_sample['VH'], epset_sample1['JL']))
    mutualInfo_VaCDR3a[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['VL'], a_s))
    mutualInfo_JaCDR3a[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['JL'], a_s))
    mutualInfo_VbCDR3a[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['VH'], a_s))
    mutualInfo_JbCDR3a[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['JH'], a_s))
    mutualInfo_VaCDR3b[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['VL'], b_s))
    mutualInfo_JaCDR3b[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['JL'], b_s))
    mutualInfo_VbCDR3b[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['VH'], b_s))
    mutualInfo_JbCDR3b[ep+'_shuffle'][s_size].append(cdr3_with_cat_MI(epset_sample['JH'], b_s))

all_MI[ep] = save_gene_MI(mutualInfo_VaVb, all_MI[ep], 'Va-Vb')
all_MI[ep] = save_gene_MI(mutualInfo_JaJb, all_MI[ep], 'Ja-Jb')
all_MI[ep] = save_gene_MI(mutualInfo_VaJa, all_MI[ep], 'Va-Ja')
all_MI[ep] = save_gene_MI(mutualInfo_VbJb, all_MI[ep], 'Vb-Jb')
all_MI[ep] = save_gene_MI(mutualInfo_VaJb, all_MI[ep], 'Va-Jb')
all_MI[ep] = save_gene_MI(mutualInfo_VbJa, all_MI[ep], 'Vb-Ja')


all_MI[ep], gene_cdr3_results['Va-CDR3a'] = save_gene_cdr3_MI(mutualInfo_VaCDR3a, 
                                            all_MI[ep], gene_cdr3_results['Va-CDR3a'], 'Va-CDR3a', ep)
all_MI[ep], gene_cdr3_results['Vb-CDR3b'] = save_gene_cdr3_MI(mutualInfo_VbCDR3b, 
                                            all_MI[ep], gene_cdr3_results['Vb-CDR3b'], 'Vb-CDR3b', ep)
all_MI[ep], gene_cdr3_results['Ja-CDR3a'] = save_gene_cdr3_MI(mutualInfo_JaCDR3a, 
                                            all_MI[ep], gene_cdr3_results['Ja-CDR3a'], 'Ja-CDR3a', ep)
all_MI[ep], gene_cdr3_results['Jb-CDR3b'] = save_gene_cdr3_MI(mutualInfo_JbCDR3b, 
                                            all_MI[ep], gene_cdr3_results['Jb-CDR3b'], 'Jb-CDR3b', ep)

all_MI[ep], gene_cdr3_results['Va-CDR3b'] = save_gene_cdr3_MI(mutualInfo_VaCDR3b, 
                                            all_MI[ep], gene_cdr3_results['Va-CDR3b'], 'Va-CDR3b', ep)
all_MI[ep], gene_cdr3_results['Vb-CDR3a'] = save_gene_cdr3_MI(mutualInfo_VbCDR3a, 
                                            all_MI[ep], gene_cdr3_results['Vb-CDR3a'], 'Vb-CDR3a', ep)
all_MI[ep], gene_cdr3_results['Ja-CDR3b'] = save_gene_cdr3_MI(mutualInfo_JaCDR3b, 
                                            all_MI[ep], gene_cdr3_results['Ja-CDR3b'], 'Ja-CDR3b', ep)
all_MI[ep], gene_cdr3_results['Jb-CDR3a'] = save_gene_cdr3_MI(mutualInfo_JbCDR3a, 
                                            all_MI[ep], gene_cdr3_results['Jb-CDR3a'], 'Jb-CDR3a', ep)

Tanno::A1::naive 835
2
5
10
15
20
25
35
50
80
100
150
200
300
500


In [26]:
for ep in all_MI.keys():
    all_MI[ep].to_csv('data/output/mutual_info/mutual_info_' + ep + '_endpadding.csv')

In [27]:
for stringtype in gene_cdr3_results:
    gene_cdr3_results[stringtype].to_csv('data/output/mutual_info/' + stringtype + '_mutual_info_by_pos_tanno_endpadding.csv')